In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [12]:
df= pd.read_csv('house_price_full.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [13]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [14]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [15]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [16]:
from tensorflow import keras

In [17]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [18]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [19]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 137.7492
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: 43.5614
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - loss: 8.6044
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 1.4365
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 0.3745
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 0.1882
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 0.1647
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - loss: 0.1539
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - loss: 0.1615
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - loss: 0.1628


In [20]:
model.predict(X)[:,0]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([12.856819 , 13.531885 , 13.019035 , 13.0370865, 13.120731 ,
       12.588133 , 12.738979 , 13.292982 , 13.142567 , 13.01455  ,
       12.960663 , 13.335176 , 13.118857 , 12.782579 , 13.407795 ,
       12.815651 , 12.922213 , 13.371678 , 12.865505 , 12.809685 ,
       13.097096 , 12.995984 , 12.971462 , 13.297086 , 13.035462 ,
       13.059931 , 13.126021 , 13.182065 , 13.391183 , 13.181755 ,
       13.025047 , 13.323322 , 13.091299 , 13.3153305, 13.463814 ,
       13.31711  , 12.561284 , 12.998097 , 13.026802 , 13.133142 ,
       12.9249935, 13.003971 , 12.578104 , 12.9768305, 12.695282 ,
       12.803699 , 13.029381 , 13.019035 , 12.836364 , 13.408432 ,
       13.292982 , 13.249671 , 13.206373 , 12.984844 , 13.26923  ,
       13.115876 , 13.179768 , 13.533418 , 12.851001 , 12.82323  ,
       13.408432 , 13.199716 , 13.297086 , 13.168083 , 13.335176 ,
       12.594792 , 12.944309 , 12.758125 , 13.102532 , 13.087296 ,
       13.255223 , 13.236877 , 13.200276 , 13.011222 , 12.8539